In [491]:
import pandas as pd
from scipy.stats import poisson
import numpy as np
import math
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

# UTILS

In [492]:
def bound_norm_random(low, high):
    res = np.random.normal((high+low)/2,(high-low)/6)
    if (res<low or res>high):
        res = bound_norm_random(low, high)
    return res

# TYPE/PHASE

In [493]:
EXPERIMENT_TYPES = ['1 off run', 'Monte Carlo', 'Monte Carlo Parameter Sweep', 'Monte Carlo Pairwise']

experiment_type = EXPERIMENT_TYPES[1]
monte_carlo_runs = 100

#correct number of runs if inconsistent with experiment type
if (experiment_type == EXPERIMENT_TYPES[0]):
    monte_carlo_runs = 1

# TIMESCALE

In [494]:
SECOND = 1
MINUTE = 60*SECOND
HOUR = 60*MINUTE
DAY = 24*HOUR
DURATION_OF_A_STEP = 1*DAY

experiment_steps = 1000
experiment_duration = experiment_steps * DURATION_OF_A_STEP
time_array = np.arange(0,experiment_steps*DURATION_OF_A_STEP,DURATION_OF_A_STEP)

# MECHANISMS (dimension)

In [495]:
mechanisms_names = ['mech_one', 'mech_two', 'mech_three']

# STATES (dimension)

In [496]:
states_names = ['a', 'b', 'c']
states_data = [[np.zeros(experiment_steps)]*len(states_names)]*monte_carlo_runs
states_data = np.zeros((monte_carlo_runs, experiment_steps, len(states_names)), dtype=int)
# states_data is a 3-dimensional array - montecarlo, time, states
# montecarlo[time[states]]

## Initial Conditions

In [497]:
states_0 = {
    'a': 0,
    'b': 0,
    'c': 300
}
# an initial condition must be set for every state
assert np.array([k in states_0 for k in states_names]).all(), 'Error: The initial condition of one or more states is unkonwn'

# copy initial condition to the states dataset
for i in range(len(states_names)):
    states_data[:,0,i] = states_0[states_names[i]]

In [498]:
T_0 = 0
time_array = T_0 + time_array

## Mechanisms Coef (params)

In [499]:
mech_one_coef_A = 0.05

# MECHANISMS EQUATIONS (func)

In [500]:
# state/mechanism matrix
def mech_one(_states_data, _time_array, _run, _step, args):
#    print('mech 1')
    _states_data[_run, _step, states_names.index('a')] += (1-mech_one_coef_A)*args[0]
    _states_data[_run, _step, states_names.index('b')] += mech_one_coef_A*args[0]
    return _states_data

def mech_two(_states_data, _time_array, _run, _step, args):
#    print('mech 2')
    _states_data[_run, _step, states_names.index('a')] -= args[0]
    return _states_data

def mech_three(_states_data, _time_array, _run, _step, args):
#    print('mech 3')
    _states_data[_run, _step, states_names.index('b')] -= args[0]
    return _states_data

def mech_four(_states_data, _time_array, _run, _step):
#    print('mech 4')
    _states_data[_run, _step, states_names.index('a')] = _states_data[_run, _step-1, states_names.index('a')]
    _states_data[_run, _step, states_names.index('b')] = _states_data[_run, _step-1, states_names.index('b')] 
    return _states_data

mechanisms = [eval(m) for m in mechanisms_names]

## Behavioral Model Coef (params) 

In [501]:
behavior_one_coef_A = 0.01
behavior_one_coef_B = -0.01

behavior_two_coef_A = 1

# BEHAVIORAL MODEL (func)

In [502]:
behaviors_names = ['behavior_one', 'behavior_two']
def behavior_one(_states_data, _time_array, _run, _step):    
    c_var = (  _states_data[_run, _step, states_names.index('c')]
             - _states_data[_run, _step-1, states_names.index('c')] )
    c_var_perc = c_var / _states_data[_run, _step-1, states_names.index('c')]
    
    if (c_var_perc > behavior_one_coef_A):
        return mech_one(_states_data, _time_array, _run, _step, [c_var])
    elif (c_var_perc < behavior_one_coef_B):
        return mech_two(_states_data, _time_array, _run, _step, [-c_var])
    return _states_data

def behavior_two(_states_data, _time_array, _run, _step):
    b_balance = _states_data[_run, _step-1, states_names.index('b')]
    if (b_balance > behavior_two_coef_A):
        return mech_three(_states_data, _time_array, _run, _step, [b_balance])
    return _states_data

behaviors = [eval(b) for b in behaviors_names]        

# ENVIRONMENTAL PROCESS (dimension)

In [503]:
env_proc_names = ['proc_one']

## Stochastic Process Coef (params)

In [504]:
proc_one_coef_A = 0.7
proc_one_coef_B = 1.3

# ENVIRONMENTAL PROCESS (func)

In [505]:
def proc_one(_states_data, _time_array, _run, _step):
    _states_data[_run, _step, states_names.index('a')] = _states_data[_run, _step-1, states_names.index('a')]
    _states_data[_run, _step, states_names.index('b')] = _states_data[_run, _step-1, states_names.index('b')] 
    _states_data[_run, _step, states_names.index('c')] = (  _states_data[_run, _step-1, states_names.index('c')]
                                                         * bound_norm_random(proc_one_coef_A, proc_one_coef_B) )
    return _states_data

env_proc = [eval(p) for p in env_proc_names]

# ENGINE

In [506]:
for i in range(monte_carlo_runs):
    for t in range(1,experiment_steps):
        for p in env_proc:
            states_data = p(_states_data=states_data,
                            _time_array=time_array, 
                            _run=i, 
                            _step=t)
        for b in behaviors:
            states_data = b(_states_data=states_data,
                            _time_array=time_array, 
                            _run=i, 
                            _step=t) #behaviors have access to exogenous data @ step-1, not @ step

/Users/markusbkoch/.local/share/virtualenvs/DiffyQ-SimCAD-4_qpgnP9/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in long_scalars
  """


# DATA COLLECTION

In [507]:
data = pd.DataFrame(states_data[0], 
                    index=[[0]*experiment_steps, time_array], 
                    columns=states_names)
for i in range(1,monte_carlo_runs):
    b = pd.DataFrame(states_data[i],
                     index=[[i]*experiment_steps, time_array], 
                     columns=states_names)
    data = data.append(b)
data

a  b    c
0  0           0  0  300
   86400      -7  0  293
   172800    -20  0  280
   259200    -86  0  214
   345600    -67  1  234
   432000    -67  1  233
   518400    -91  1  209
   604800    -99  1  201
   691200    -93  1  207
   777600   -104  1  196
   864000   -101  1  199
   950400    -86  1  214
   1036800   -92  1  208
   1123200   -92  1  210
   1209600   -99  1  203
   1296000  -122  1  180
   1382400  -135  1  167
   1468800  -161  1  141
   1555200  -161  1  141
   1641600  -187  1  115
   1728000  -195  1  107
   1814400  -201  1  101
   1900800  -189  1  113
   1987200  -189  1  112
   2073600  -189  1  111
   2160000  -191  1  109
   2246400  -200  1  100
   2332800  -206  1   94
   2419200  -206  1   94
   2505600  -218  1   82
...          ... ..  ...
99 83808000 -386  0    0
   83894400 -386  0    0
   83980800 -386  0    0
   84067200 -386  0    0
   84153600 -386  0    0
   84240000 -386  0    0
   84326400 -386  0    0
   84412800 -386  0    0
   84499200 -386  0    0
   84585600 -386  0    0
   84672000 -386  0    0
   84758400 -386  0    0
   84844800 -386  0    0
   84931200 -386  0    0
   85017600 -386  0    0
   85104000 -386  0    0
   85190400 -386  0    0
   85276800 -386  0    0
   85363200 -386  0    0
   85449600 -386  0    0
   85536000 -386  0    0
   85622400 -386  0    0
   85708800 -386  0    0
   85795200 -386  0    0
   85881600 -386  0    0
   85968000 -386  0    0
   86054400 -386  0    0
   86140800 -386  0    0
   86227200 -386  0    0
   86313600 -386  0    0

[100000 rows x 3 columns]